In [57]:
import re
import json

code = """

const AAA: int = 1

def func(a: int) -> int:
    a = a + 1
    return a

def main() -> None:
    var a: int = 2
    var b: string = "eiwf23h32ur"
    
    var c: int = 12
    del a
    var a: int = 1
    
    c = a + 10
    var x: int = func(c) + 1
    var v: int = (c + a) // 10

    var xx: int = 10 + 1
    xx = 1 + 10
"""

KEYS = ["const", "var", "def", "del", "return"]
DATA_TYPES = ["int", "string", "None"]

class SymbolTable:
    def __init__(self):
        self.symbols = {}
        self.variable_versions = {}
    
    def add_symbol(self, name, key, var_type, value=None, pointer=None, is_constant=False):
        version = 1
        if key == "var" and name in self.variable_versions:
            version = self.variable_versions[name] + 1
            self.variable_versions[name] = version
        elif name not in self.variable_versions:
            self.variable_versions[name] = 1
        
        symbol_id = name
        
        self.symbols[symbol_id] = {
            "name": name,
            "key": key,
            "type": var_type,
            "value": value,
            "pointer": pointer or f"{key}_{name}_ptr_{id(self)}_{version}",
            "version": version,
            "status": "active",
            "id": symbol_id
        }
        
        if is_constant:
            self.symbols[symbol_id]["key"] = "const"
        
        return symbol_id

    def get_active_symbol(self, name):
        if name not in self.symbols:
            return None
        
        symbol = self.symbols[name]
        if symbol["status"] == "active":
            return symbol
        
        return None

    def delete_symbol(self, name):
        if name in self.symbols:
            self.symbols[name]["status"] = "deleted"
            return True
        return False

class Parser:
    def __init__(self):
        self.scopes = []
        self.current_scope_index = 0
        self.symbol_counter = 0
    
    def generate_pointer(self, prefix):
        self.symbol_counter += 1
        return f"{prefix}_ptr_{self.symbol_counter:03d}"
    
    def parse_code(self, code: str):
        # Очистка комментариев
        code = re.sub(r"#.*", "", code)
        code = re.sub(r"'''.*?'''", "", code, flags=re.DOTALL)
        code = re.sub(r'""".*?"""', "", code, flags=re.DOTALL)
        
        lines = code.split("\n")
        cleaned_lines = []
        
        # Сохраняем отступы
        for line in lines:
            if line.strip():  # Пропускаем пустые строки
                cleaned_lines.append(line)
        
        # Инициализируем глобальную область
        global_scope = {
            "level": 0,
            "type": "module",
            "local_variables": [],
            "graph": [],
            "symbol_table": SymbolTable()
        }
        self.scopes.append(global_scope)
        
        i = 0
        while i < len(cleaned_lines):
            line = cleaned_lines[i]
            indent = len(line) - len(line.lstrip())
            line_content = line.strip()
            
            # Определяем, в каком scope мы находимся
            current_scope = self.get_current_scope(indent)
            
            if not current_scope:
                current_scope = self.scopes[0]  # По умолчанию глобальная область
            
            # Парсим строку в соответствующем scope
            if current_scope["type"] == "module":
                self.parse_global_line(line_content, current_scope, cleaned_lines, i)
            else:  # function scope
                self.parse_function_line(line_content, current_scope)
            
            i += 1
        
        # Конвертируем SymbolTable в dict
        for scope in self.scopes:
            scope["symbol_table"] = scope["symbol_table"].symbols
        
        return self.scopes
    
    def get_current_scope(self, indent):
        """Определяет текущий scope на основе отступа"""
        if indent == 0:
            return self.scopes[0]  # Глобальная область
        
        # Ищем самую глубокую функцию
        for scope in reversed(self.scopes):
            if scope["type"] == "function":
                return scope
        
        return self.scopes[0]
    
    def parse_global_line(self, line: str, scope: dict, all_lines: list, current_index: int):
        """Парсит строку в глобальной области видимости"""
        if not line:
            return
        
        for key in KEYS:
            if line.startswith(key + " ") or line == key:
                if key == "const":
                    self.parse_const(line, scope)
                elif key == "var":
                    self.parse_var(line, scope)
                elif key == "def":
                    self.parse_function_declaration(line, scope, all_lines, current_index)
                return
        
        # В глобальной области только объявления
        print(f"Warning: Unexpected line in global scope: {line}")
    
    def parse_function_line(self, line: str, scope: dict):
        """Парсит строку внутри функции"""
        if not line:
            return
        
        parsed = False
        
        for key in KEYS:
            if line.startswith(key + " ") or line == key:
                if key == "const":
                    parsed = self.parse_const(line, scope)
                elif key == "var":
                    parsed = self.parse_var(line, scope)
                elif key == "def":
                    # Вложенные функции пока не поддерживаем
                    parsed = False
                elif key == "del":
                    parsed = self.parse_delete(line, scope)
                elif key == "return":
                    parsed = self.parse_return(line, scope)
                break
        
        if not parsed:
            if re.match(r'[a-zA-Z_][a-zA-Z0-9_]*\s*\(', line) and 'var ' in line:
                parsed = self.parse_function_call_assignment(line, scope)
            elif re.match(r'[a-zA-Z_][a-zA-Z0-9_]*\s*\(', line):
                parsed = self.parse_function_call(line, scope)
            elif "=" in line and not any(line.startswith(k + " ") for k in ["const", "var", "def"]):
                parsed = self.parse_assignment(line, scope)
            elif "+=" in line or "-=" in line or "*=" in line or "/=" in line:
                parsed = self.parse_augmented_assignment(line, scope)
    
    def parse_function_declaration(self, line: str, global_scope: dict, all_lines: list, current_index: int):
        """Обрабатывает объявление функции в глобальной области"""
        pattern = r"def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\((.*?)\)\s*(?:->\s*([a-zA-Z_][a-zA-Z0-9_]*))?\s*:"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        func_name, params_str, return_type = match.groups()
        return_type = return_type if return_type else "None"
        
        # Парсим параметры
        parameters = []
        if params_str.strip():
            param_pattern = r'([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)'
            params = re.findall(param_pattern, params_str)
            for param_name, param_type in params:
                parameters.append({
                    "name": param_name,
                    "type": param_type
                })
        
        # Добавляем функцию в глобальную таблицу символов
        symbol_id = global_scope["symbol_table"].add_symbol(
            name=func_name,
            key="function",
            var_type="function",
            pointer=self.generate_pointer(f"func_{func_name}")
        )
        
        global_scope["graph"].append({
            "node": "function_declaration",
            "content": line,
            "function_name": func_name,
            "symbol_id": symbol_id,
            "parameters": parameters,
            "return_type": return_type,
            "body_level": 1
        })
        
        # Создаем новую область видимости для функции
        func_scope = {
            "level": 1,
            "type": "function",
            "parent_scope": 0,
            "function_name": func_name,
            "parameters": parameters,
            "return_type": return_type,
            "local_variables": [],
            "graph": [],
            "symbol_table": SymbolTable(),
            "return_info": {
                "has_return": False,
                "return_value": None,
                "return_type": return_type
            }
        }
        
        # Добавляем параметры в таблицу символов функции
        for param in parameters:
            func_scope["symbol_table"].add_symbol(
                name=param["name"],
                key="var",
                var_type=param["type"]
            )
            func_scope["local_variables"].append(param["name"])
        
        # Добавляем scope функции в список
        self.scopes.append(func_scope)
        
        return True
    
    def parse_const(self, line: str, scope: dict):
        pattern = r"const\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*(.+)"
        match = re.match(pattern, line)
        
        if match:
            name, var_type, value = match.groups()
            value = self.clean_value(value)
            
            symbol_id = scope["symbol_table"].add_symbol(
                name=name,
                key="const",
                var_type=var_type,
                value=value,
                is_constant=True
            )
            
            scope["local_variables"].append(symbol_id)
            
            scope["graph"].append({
                "node": "declaration",
                "content": line,
                "symbols": [symbol_id],
                "operations": [
                    {"type": "NEW_CONST", "target": symbol_id, "const_type": var_type},
                    {"type": "ASSIGN", "target": symbol_id, "value": value}
                ]
            })
            
            return True
        return False
    
    def parse_var(self, line: str, scope: dict):
        pattern = r"var\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*(.+)"
        match = re.match(pattern, line)
        
        if match:
            name, var_type, value = match.groups()
            value = self.clean_value(value)
            
            symbol_id = scope["symbol_table"].add_symbol(
                name=name,
                key="var",
                var_type=var_type,
                value=value
            )
            
            scope["local_variables"].append(symbol_id)
            
            scope["graph"].append({
                "node": "declaration",
                "content": line,
                "symbols": [symbol_id],
                "operations": [
                    {"type": "NEW_VAR", "target": symbol_id, "var_type": var_type},
                    {"type": "ASSIGN", "target": symbol_id, "value": value}
                ]
            })
            
            return True
        return False
    
    def parse_delete(self, line: str, scope: dict):
        pattern = r"del\s+([a-zA-Z_][a-zA-Z0-9_]*)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        name = match.group(1)
        
        deleted = scope["symbol_table"].delete_symbol(name)
        
        if deleted:
            scope["graph"].append({
                "node": "delete",
                "content": line,
                "symbols": [name],
                "operations": [
                    {"type": "DELETE", "target": name}
                ]
            })
        
        return deleted
    
    def parse_return(self, line: str, scope: dict):
        pattern = r"return\s+(.+)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        value = match.group(1).strip()
        
        dependencies = []
        var_pattern = r'([a-zA-Z_][a-zA-Z0-9_]*)'
        vars_in_value = re.findall(var_pattern, value)
        for var in vars_in_value:
            if var not in KEYS and var not in DATA_TYPES:
                dependencies.append(var)
        
        scope["graph"].append({
            "node": "return",
            "content": line,
            "symbols": [value] if value.isalpha() else [],
            "operations": [
                {"type": "RETURN", "value": value}
            ],
            "dependencies": dependencies
        })
        
        if "return_info" in scope:
            scope["return_info"]["has_return"] = True
            scope["return_info"]["return_value"] = value
        
        return True
    
    def parse_assignment(self, line: str, scope: dict):
        pattern = r"([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*(.+)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        name, expression = match.groups()
        
        symbol = scope["symbol_table"].get_active_symbol(name)
        if not symbol:
            return False  # Переменная должна быть объявлена
        
        operations = []
        dependencies = []
        
        if "+" in expression:
            parts = expression.split("+")
            if len(parts) == 2:
                left, right = parts[0].strip(), parts[1].strip()
                operations.append({
                    "type": "BINARY_OP",
                    "target": name,
                    "operator": "+",
                    "left": left,
                    "right": right
                })
                
                if left.isalpha() and left not in KEYS and left not in DATA_TYPES:
                    dependencies.append(left)
                if right.isalpha() and right not in KEYS and right not in DATA_TYPES:
                    dependencies.append(right)
        else:
            operations.append({
                "type": "ASSIGN",
                "target": name,
                "value": self.clean_value(expression)
            })
        
        # Обновляем значение переменной
        scope["symbol_table"].add_symbol(
            name=name,
            key="var",
            var_type=symbol["type"],
            value=expression
        )
        
        scope["graph"].append({
            "node": "assignment",
            "content": line,
            "symbols": [name],
            "operations": operations,
            "dependencies": dependencies
        })
        
        return True
    
    def parse_augmented_assignment(self, line: str, scope: dict):
        pattern = r"([a-zA-Z_][a-zA-Z0-9_]*)\s*(\+=|-=|\*=|\/=)\s*(.+)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        name, operator, value = match.groups()
        
        symbol = scope["symbol_table"].get_active_symbol(name)
        if not symbol:
            return False
        
        operator_map = {
            "+=": "ADD",
            "-=": "SUB",
            "*=": "MUL",
            "/=": "DIV"
        }
        
        op_type = operator_map.get(operator, "ADD")
        
        operations = [{
            "type": "AUGMENTED_ASSIGN",
            "target": name,
            "operator": op_type,
            "value": value
        }]
        
        dependencies = []
        if value.isalpha() and value not in KEYS and value not in DATA_TYPES:
            dependencies.append(value)
        
        # Обновляем значение переменной
        scope["symbol_table"].add_symbol(
            name=name,
            key="var",
            var_type=symbol["type"],
            value=f"{name} {operator} {value}"
        )
        
        scope["graph"].append({
            "node": "augmented_assignment",
            "content": line,
            "symbols": [name],
            "operations": operations,
            "dependencies": dependencies
        })
        
        return True
    
    def parse_function_call(self, line: str, scope: dict):
        pattern = r"([a-zA-Z_][a-zA-Z0-9_]*)\s*\((.*?)\)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        func_name, args_str = match.groups()
        args = []
        if args_str.strip():
            args = [arg.strip() for arg in args_str.split(',')]
        
        operations = [{
            "type": "FUNCTION_CALL",
            "function": func_name,
            "arguments": args
        }]
        
        dependencies = []
        for arg in args:
            if arg.isalpha() and arg not in KEYS and arg not in DATA_TYPES:
                dependencies.append(arg)
        
        scope["graph"].append({
            "node": "function_call",
            "content": line,
            "function": func_name,
            "arguments": args,
            "operations": operations,
            "dependencies": dependencies
        })
        
        return True
    
    def parse_function_call_assignment(self, line: str, scope: dict):
        pattern = r"var\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*\((.*?)\)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        var_name, var_type, func_name, args_str = match.groups()
        args = []
        if args_str.strip():
            args = [arg.strip() for arg in args_str.split(',')]
        
        # Добавляем переменную
        symbol_id = scope["symbol_table"].add_symbol(
            name=var_name,
            key="var",
            var_type=var_type
        )
        
        scope["local_variables"].append(symbol_id)
        
        operations = [
            {"type": "NEW_VAR", "target": var_name, "var_type": var_type},
            {"type": "FUNCTION_CALL_ASSIGN", "function": func_name, "arguments": args, "target": var_name}
        ]
        
        dependencies = []
        for arg in args:
            if arg.isalpha() and arg not in KEYS and arg not in DATA_TYPES:
                dependencies.append(arg)
        
        scope["graph"].append({
            "node": "function_call_assignment",
            "content": line,
            "symbols": [var_name],
            "function": func_name,
            "arguments": args,
            "operations": operations,
            "dependencies": dependencies
        })
        
        return True
    
    def clean_value(self, value: str):
        value = value.strip()
        if value.startswith('"') and value.endswith('"'):
            return value
        elif value.isdigit():
            return int(value)
        return value

def main():
    parser = Parser()
    result = parser.parse_code(code)
    
    json_output = json.dumps(result, indent=2, default=str)
    print(json_output)
    
    with open("parsed_code.json", "w") as f:
        f.write(json_output)
    
    return result

if __name__ == "__main__":
    main()

[
  {
    "level": 0,
    "type": "module",
    "local_variables": [
      "AAA"
    ],
    "graph": [
      {
        "node": "declaration",
        "content": "const AAA: int = 1",
        "symbols": [
          "AAA"
        ],
        "operations": [
          {
            "type": "NEW_CONST",
            "target": "AAA",
            "const_type": "int"
          },
          {
            "type": "ASSIGN",
            "target": "AAA",
            "value": 1
          }
        ]
      },
      {
        "node": "function_declaration",
        "content": "def func(a: int) -> int:",
        "function_name": "func",
        "symbol_id": "func",
        "parameters": [
          {
            "name": "a",
            "type": "int"
          }
        ],
        "return_type": "int",
        "body_level": 1
      },
      {
        "node": "function_declaration",
        "content": "def main() -> None:",
        "function_name": "main",
        "symbol_id": "main",
        "parameters

const a: int = 1
var b: string = "hello"

var c: int = 1 + 2

In [ ]:
a = {
    "key": "const",
    "name": "a",
    "type": "int",
    "pointer": "efij213e32irh"
}

b = {
    "key": "var",
    "name": "b",
    "type": "string",
    "pointer": "askf21e128ndj"
}

In [ ]:
[
    {
        "1": {
            "local_variables": [a, b, c, d],
            "graph": [
                "NEW c",
                "SUMM c, (1, 2)"
            ]
        },
    }
]

const CONSTANT: int = 1

def main():
    var a: int = 2
    var b: string = "eiwf23h32ur"

    var c = 12
    del a
    var a: int = 1

    c = a + 10
    return c

const ANOTHER_CONST: string = "hello"